<div style="
    color: blue; /* Text color */
    background-color: #f0f0f0; /* Light grey background */
    font-family: 'Pacifico', cursive; /* Stylish font */
    font-size: 40px; /* Font size */
    font-weight: bold; /* Bold text */
    padding: 20px; /* Padding around the text */
    border: 2px solid #ccc; /* Border with a light grey color */
    border-radius: 10px; /* Rounded corners */
    text-align: center; /* Centered text */
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); /* Subtle shadow */
    position: relative; /* Required for pseudo-elements positioning */
    display: inline-block; /* Inline-block for proper spacing */
">
    <span style="
        position: absolute; /* Absolute positioning */
        left: -40px; /* Positioning arrow on the left */
        top: 50%; /* Vertically center */
        transform: translateY(-50%); /* Adjust vertical alignment */
        font-size: 30px; /* Arrow size */
        color: red; /* Arrow color */
    ">➤</span>
    NN Modeling
    <span style="
        position: absolute; /* Absolute positioning */
        right: -40px; /* Positioning arrow on the right */
        top: 50%; /* Vertically center */
        transform: translateY(-50%); /* Adjust vertical alignment */
        font-size: 30px; /* Arrow size */
        color: red; /* Arrow color */
    ">➤</span>
</div>

<link href="https://fonts.googleapis.com/css2?family=Pacifico&display=swap" rel="stylesheet">


In [9]:
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module="sklearn")
prediction_days_count = 2

def get_X_Y (exception_countries = []):
    df = pd.read_csv("../Data/Raw/new_approach/Weekly_Covid_Data.csv")

    # delete called countries:
    for iso_code in exception_countries:
        df = df[df["iso_code"] != iso_code]
    df = df.reset_index(drop=True)
    
    country_num = int(len(df["iso_code"].unique()))
    
    df.drop(["iso_code", "week_no"], axis=1, inplace=True)
    #df.drop(columns=df.columns[0:1], axis=1, inplace=True)
    pop = df.iloc[0]["population"]
    last_country = df.iloc[country_num*176 - 1]
    print(last_country)
    cnt = df["population"].value_counts()
    
    # now we want to join datas together
    features_list = []
    X_list = []
    Y_list = []
    
    for i in range(country_num):  # 234 is number of countries
        for j in range(176):  # 176 is number of weeks we have
            row_list = df.loc[i * 176 + j, :].values.flatten().tolist()
            features_list.append(row_list)
    
    for i in range((len(features_list) - prediction_days_count) + 1):
        fl = features_list[i]
        fl_len = len(features_list[i])
        this_population = features_list[i][fl_len - 1]
        for j in range(1, prediction_days_count):
            if this_population == features_list[i + j][fl_len - 1]:
                fl.extend(features_list[i + j])
            else:
                this_population = -1
                break
        if this_population != -1:
            X_list.append(fl)
            Y_list.append(features_list[(i + prediction_days_count) - 1][0])
        
    return X_list, Y_list, country_num

In [83]:
import math

def train_test_splitter(X, Y, country_num, prediction_days_count, batch_count, batch_length):
    xlen = len(X)
    xs_count_for_each_country = int(xlen / country_num)
    X_train = []
    X_test = []
    Y_train = []
    Y_test = []
    steps = math.floor((xs_count_for_each_country - (batch_count * batch_length)) / (batch_count + 1)) 
    i = 0
    for i in range(country_num):
        j = 0
        for j in range((i * xs_count_for_each_country), (i * xs_count_for_each_country) + steps):
            X_train.append(X[j])
            Y_train.append(Y[j])
        while(j + batch_length + steps < (i+1) * xs_count_for_each_country):
            for k in range(j + prediction_days_count, j + batch_length - prediction_days_count):
                X_test.append(X[k])
                Y_test.append(Y[k])
            j += batch_length
            for k in range(j, j + steps):
                X_train.append(X[k])
                Y_train.append(Y[k])
            j += steps + 1
            while j < (i+1) * xs_count_for_each_country:
                X_train.append(X[j])
                Y_train.append(Y[j])
                j += 1
    return X_train, X_test, Y_train, Y_test

In [84]:
# getting data:
X_list, Y_list, country_num = get_X_Y(["CHN"])

new_cases                     7.500000e+01
new_deaths                    2.000000e+00
new_vaccinations              0.000000e+00
new_people_vaccinated         0.000000e+00
reproduction_rate             9.500000e-01
stringency_index              8.796000e+01
excess_mortality             -1.000000e+03
population_density            4.272900e+01
median_age                    1.960000e+01
aged_65_older                 2.822000e+00
aged_70_older                 1.882000e+00
cardiovasc_death_rate         3.078460e+02
diabetes_prevalence           1.820000e+00
female_smokers                1.600000e+00
male_smokers                  3.070000e+01
hospital_beds_per_thousand    1.700000e+00
life_expectancy               6.149000e+01
human_development_index       5.710000e-01
population                    1.632054e+07
Name: 41007, dtype: float64


In [85]:
x_df = pd.DataFrame(X_list)
min_max_scaler_X = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler_X.fit_transform(x_df)

y_df = pd.DataFrame(Y_list)
min_max_scaler_Y = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler_Y.fit_transform(y_df)

X_scaled_list = x_scaled.tolist()
Y_scaled_list = y_scaled.tolist()

In [86]:
X_train, X_test, Y_train, Y_test = train_test_splitter(X_scaled_list, Y_scaled_list,
                                                       country_num=country_num,
                                                       prediction_days_count=prediction_days_count,
                                                       batch_count=3,
                                                       batch_length=10)
X_tr = np.array(X_train)
y_tr = np.array(Y_train).reshape(len(Y_train))
X_te = np.array(X_test)
y_te = np.array(Y_test).reshape(len(Y_test))

In [87]:
min_val = min_max_scaler_Y.data_min_[0]
max_val = min_max_scaler_Y.data_max_[0]

In [99]:
from tensorflow.keras import backend as K

# Define the custom loss function


# Define the custom loss function
def rescale_data_tensor(data):
    return (data * (max_val - min_val) + min_val)/10 # you can delete /10 I put it because we add 10 people
                                                     # error 
    
def loss_function(y_true, y_pred):
    y_true = rescale_data_tensor(y_true)
    y_pred = rescale_data_tensor(y_pred)
    return K.sqrt(K.mean(K.square(y_true - y_pred)))

# First Model

In [100]:
modle_1 = keras.Sequential([
    keras.Input(shape=(X_tr.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)  # No activation function for regression
])

In [101]:
modle_1.compile(optimizer='adam',
              loss=loss_function)

In [102]:
modle_1.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_18 (Dense)                │ (None, 64)             │         2,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,561 (10.00 KB)

 Trainable params: 2,561 (10.00 KB)

 Non-trainable params: 0 (0.00 B)

In [103]:
history = modle_1.fit(X_tr, y_tr, validation_split=0.25, epochs=50, batch_size=50)

Epoch 1/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 2s 930us/step - loss: 19538.1309 - val_loss: 9768.6650
Epoch 2/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - loss: 3962.0684 - val_loss: 8826.1230
Epoch 3/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - loss: 3047.3418 - val_loss: 8629.0020
Epoch 4/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 2428.6196 - val_loss: 8976.9238
Epoch 5/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 2693.8110 - val_loss: 7920.3096
Epoch 6/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step - loss: 2870.8452 - val_loss: 6905.6445
Epoch 7/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step - loss: 2515.3303 - val_loss: 10402.8145
Epoch 8/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 713us/step - loss: 3558.6689 - val_loss: 6593.0298
Epoch 9/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 711us/step - loss: 2223.8696 - val_loss: 6650.5957
Epoch 10/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - loss: 2641.7146 - val_loss: 7626.2295
Epoch 11/50
577/577 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - 

In [104]:
y_pred = modle_1.predict (X_te)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [105]:
loss_function(y_te, y_pred)

<tf.Tensor: shape=(), dtype=float64, numpy=7658.91743214217>

# Second Model

In [106]:
model_2 = keras.Sequential([
    keras.Input(shape=(X_tr.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)  # No activation function for regression
])

In [107]:
model_2.compile(optimizer='adam',
              loss=loss_function)

In [108]:
model_2.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 128)            │         4,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,361 (60.00 KB)

 Trainable params: 15,361 (60.00 KB)

 Non-trainable params: 0 (0.00 B)

In [109]:
history = model_2.fit(X_tr, y_tr, validation_split=0.25, epochs=100, batch_size=32)

Epoch 1/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 10132.6973 - val_loss: 5917.3799
Epoch 2/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 822us/step - loss: 2826.5164 - val_loss: 6077.2837
Epoch 3/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 826us/step - loss: 2293.6726 - val_loss: 2592.9077
Epoch 4/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 844us/step - loss: 1711.3815 - val_loss: 1853.5065
Epoch 5/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 948us/step - loss: 1567.0398 - val_loss: 2212.0449
Epoch 6/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 836us/step - loss: 1291.7601 - val_loss: 4934.8613
Epoch 7/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - loss: 1457.3086 - val_loss: 1613.1071
Epoch 8/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 838us/step - loss: 1021.5228 - val_loss: 1743.0806
Epoch 9/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 812us/step - loss: 926.3903 - val_loss: 1350.9608
Epoch 10/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 800us/step - loss: 869.9085 - val_loss: 2575.2805
Epoch 11/100
902/902 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/s

In [111]:
y_pred = model_2.predict (X_te)
loss_function(y_te, y_pred)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=6586.99384925401>

# Third Model

In [112]:
X_tr_reshaped = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
X_te_reshaped = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)

model_3 = keras.Sequential([
    keras.Input(shape=(X_tr.shape[1], 1)),
    keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='leaky_relu'),
    keras.layers.Dense(1)  # No activation function for regression
])

In [113]:
model_3.compile(optimizer='adam',
              loss=loss_function)

In [114]:
model_3.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 36, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 18, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 16, 32)         │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 8, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,945 (89.63 KB)

 Trainable params: 22,945 (89.63 KB)

 Non-trainable params: 0 (0.00 B)

In [115]:
history = model_3.fit(X_tr_reshaped, y_tr, validation_split=0.25, epochs=50, batch_size=32)

Epoch 1/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 7368.5303 - val_loss: 3521.8357
Epoch 2/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3277.4753 - val_loss: 3314.4783
Epoch 3/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1853.2959 - val_loss: 2386.3096
Epoch 4/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1222.5406 - val_loss: 1643.0807
Epoch 5/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1079.8007 - val_loss: 2017.6349
Epoch 6/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 956.7016 - val_loss: 1146.9729
Epoch 7/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 913.5614 - val_loss: 3086.8528
Epoch 8/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1036.4404 - val_loss: 1543.5526
Epoch 9/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 890.6574 - val_loss: 982.8900
Epoch 10/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 710.0589 - val_loss: 1215.3370
Epoch 11/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 723.6082 - val_loss: 15

In [116]:
y_pred = model_3.predict (X_te)
loss_function(y_te, y_pred)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=6565.8024041816425>

# Forth Model

In [117]:
X_tr_reshaped = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
X_te_reshaped = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)

model_4 = keras.Sequential([
    keras.Input(shape=(X_tr.shape[1], 1)),
    keras.layers.SimpleRNN(64, activation='relu'),
    keras.layers.Dense(1)  # No activation function for regression
])

In [119]:
model_4.compile(optimizer='adam',
              loss=loss_function)

In [120]:
model_4.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,289 (16.75 KB)

 Trainable params: 4,289 (16.75 KB)

 Non-trainable params: 0 (0.00 B)

In [121]:
history = model_4.fit(X_tr_reshaped, y_tr, validation_split=0.25, epochs=50, batch_size=32)

Epoch 1/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 6715.2119 - val_loss: 5164.3228
Epoch 2/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4815.7139 - val_loss: 5750.4980
Epoch 3/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3506.1104 - val_loss: 3905.8457
Epoch 4/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 2105.2246 - val_loss: 3039.9346
Epoch 5/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1810.4463 - val_loss: 2730.8311
Epoch 6/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1653.2291 - val_loss: 2977.8538
Epoch 7/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1576.2588 - val_loss: 2869.6440
Epoch 8/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1580.3351 - val_loss: 2661.3660
Epoch 9/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1462.3433 - val_loss: 2644.5703
Epoch 10/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 1301.3262 - val_loss: 2649.3774
Epoch 11/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1433.5942 - val_lo

In [122]:
y_pred = model_4.predict (X_te)
loss_function(y_te, y_pred)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=7488.029311789402>

# Fifth Model

In [123]:
X_tr_reshaped = X_tr.reshape(X_tr.shape[0], X_tr.shape[1], 1)
X_te_reshaped = X_te.reshape(X_te.shape[0], X_te.shape[1], 1)

model_5 = keras.Sequential([
    keras.Input(shape=(X_tr.shape[1], 1)),
    keras.layers.LSTM(64, activation='relu'),
    keras.layers.Dense(1)  # No activation function for regression
])

In [124]:
model_5.compile(optimizer='adam',
              loss=loss_function)

In [125]:
model_5.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,961 (66.25 KB)

 Trainable params: 16,961 (66.25 KB)

 Non-trainable params: 0 (0.00 B)

In [126]:
history = model_5.fit(X_tr_reshaped, y_tr, validation_split=0.25, epochs=50, batch_size=32)

Epoch 1/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 5473.9897 - val_loss: 3512.6721
Epoch 2/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 5021.6865 - val_loss: 3355.4163
Epoch 3/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4162.8149 - val_loss: 3021.9329
Epoch 4/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 3479.2500 - val_loss: 2580.7573
Epoch 5/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 3310.5261 - val_loss: 2376.0908
Epoch 6/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 2302.4231 - val_loss: 2234.1448
Epoch 7/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1689.8063 - val_loss: 1331.8375
Epoch 8/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1314.3998 - val_loss: 1058.6250
Epoch 9/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1410.0973 - val_loss: 1870.1173
Epoch 10/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 1233.4760 - val_loss: 1254.5377
Epoch 11/50
902/902 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 1409.6354 - val_lo

In [131]:
y_pred = model_5.predict (X_te)
loss_function(tf.convert_to_tensor(y_te), y_pred)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


<tf.Tensor: shape=(), dtype=float64, numpy=6702.900378200025>